In [ ]:
!pip install apify-client
!pip install python-dotenv
!pip install datasets
!pip install bitsandbytes
!pip install accelerate
!pip install trl
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 485.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 36.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.4 MB/s eta

In [ ]:
import os
import gc
import pandas as pd
from datasets import load_dataset
from dotenv import load_dotenv

from apify_client import ApifyClient

from huggingface_hub import login
import torch
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments

In [ ]:
load_dotenv()

True

Fetch the post captions from ospreypacks:

In [ ]:
# Initialize the ApifyClient
client = ApifyClient(os.environ["APIFY_TOKEN"])

In [ ]:
# Run scraping (if didn't already)

# Actor input
run_input = {
    "directUrls": ["https://www.instagram.com/ospreypacks/"],
    "resultsType": "posts",
    "searchLimit": 1,
    "addParentData": False
}

# Run the Actor
run = client.actor("shu8hvrXbJbY3Eb9W").call(run_input=run_input)

# Extract post captions from the run's dataset
data = client.dataset(run["defaultDatasetId"]).iterate_items()

In [ ]:
# Retrieve previous run results (otherwise)

# Extract post captions from the dataset
data = client.dataset("Q5CDl4lmhD0vbuUwS").iterate_items()

Prepare the dataset and save it as a json file:

In [ ]:
def gen_text(row):
    return f'{row["instruction"]}\n\n{row["output"]}'

# Create pandas DataFrame
df = pd.DataFrame(columns=["output", "instruction", "text"])
df["output"] = [post["caption"] for post in data]
df["instruction"] = "Write a caption for an Instagram post that advertises a backpack."
df["text"] = df.apply(gen_text, axis=1)

# Convert DataFrame to JSON and save to a file
json_string = df.to_json("backpack_posts.json", orient="records")

In [ ]:
dataset = load_dataset("json", data_files="backpack_posts.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['text', 'output', 'instruction'],
    num_rows: 1420
})

In [ ]:
print(dataset[0]["text"])

Write a caption for an Instagram post that advertises a backpack.

A pack that has your back so you can venture forth ⛰️

#OspreyPacks | 📷 @adventures.with.hollie


Get the Gemma 2b model from HuggingFace:

In [ ]:
login(token=os.environ["HF_TOKEN"])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
model_id = "google/gemma-2b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir="output")
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config,
                                             device_map={"": 0}, cache_dir="output")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Fine tune the model using LoRA:

In [ ]:
peft_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=130,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="output",
        optim="adamw_torch"
    ),
    peft_config=peft_config
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1420 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
gc.collect()

111

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

Wed May 15 17:10:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0              26W /  70W |   5653MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
trainer.train()

Step,Training Loss
1,6.656100
2,6.575700
3,6.899800
4,5.214000
5,4.787400
6,4.359600
7,4.343200
8,3.936200
9,4.007300
10,3.223600


TrainOutput(global_step=130, training_loss=2.464160733956557, metrics={'train_runtime': 233.0673, 'train_samples_per_second': 4.462, 'train_steps_per_second': 0.558, 'total_flos': 1503232192020480.0, 'train_loss': 2.464160733956557, 'epoch': 0.7323943661971831})

Upload the model to HuggingFace:

In [ ]:
merged_model = trainer.model.merge_and_unload()
merged_model.push_to_hub("IvanBanny/gemma-backpack-model")

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.21G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/IvanBanny/gemma-backpack-model/commit/91fde4409c2ff3022a4e7190f5555cbb1dd778a6', commit_message='Upload GemmaForCausalLM', commit_description='', oid='91fde4409c2ff3022a4e7190f5555cbb1dd778a6', pr_url=None, pr_revision=None, pr_num=None)

Inference:

In [ ]:
model_reload = AutoModelForCausalLM.from_pretrained("IvanBanny/gemma-backpack-model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/3.21G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
text = "Create a new post about the 'Adventure' model backpack for 25 liters for $200, which is great for climbers."
inputs = tokenizer(text, return_tensors="pt").to("cuda:0")
outputs = model_reload.generate(**inputs, max_new_tokens=500)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Create a new post about the 'Adventure' model backpack for 25 liters for $200, which is great for climbers.

## Conquer the Mountain with the Adventure Backpack!

The Adventure backpack is the ultimate companion for any outdoor enthusiast seeking a comfortable and efficient way to carry their gear. With a robust design and a spacious interior, this backpack is built to handle the most demanding climbs and expeditions.

**Here's what makes the Adventure backpack perfect for climbers:**

* **Durable construction:** Made from high-quality materials and featuring a reinforced frame, the Adventure can handle heavy loads and extreme weather conditions.
* **Spacious interior:** The main compartment offers ample space for your climbing gear, including boots, helmets, ropes, and other essentials.
* **Multiple pockets:** A dedicated pocket for water bottles, tools, and other small items keeps your hands free for climbing.
* **Adjustable straps:** The adjustable straps ensure a comfortable fit an

In [ ]:
text = "Write me a post about the giveaway of 3 bags of the new collection."
inputs = tokenizer(text, return_tensors="pt").to("cuda:0")
outputs = model_reload.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Write me a post about the giveaway of 3 bags of the new collection.

## Get ready to shop! 🛍️

We're giving away 3 bags of our latest collection! 🎉

These bags are the perfect way to add a touch of luxury and style to your everyday life. They're made from high-quality materials and feature a unique design that will make you stand out from the crowd.

**Here's what you need to do to enter:**

1. Follow us on Instagram @your_brand_name
2. Like this post
3. Tag 3 friends in the comments who you think would love these bags

**The giveaway will run from [start date] to [end date].**

We will randomly select the winners on [date].

**Prizes:**

* 3 bags of the new collection

**To enter:**

1. Follow us on Instagram @your_brand_name
2. Like this post
3. Tag 3 friends in the comments who you think would love these bags
